In [ ]:
!pip install pygad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


# PyGAD

In [ ]:
import pygad
import string

target_passcode = "PYTHON345er"


def fitness_func(ga_instance, solution, solution_idx):
    fitness = sum(1 for i in range(len(target_passcode)) if solution[i] == ord(target_passcode[i]))
    return fitness


#alphabet = string.ascii_uppercase
alphabet = string.ascii_letters + string.digits
gene_space = [ord(char) for char in alphabet]
num_genes = len(target_passcode)

ga_instance = pygad.GA(num_generations=200,
                       num_parents_mating=5,
                       fitness_func=fitness_func,
                       sol_per_pop=20,
                       num_genes=num_genes,
                       gene_space=gene_space,
                       mutation_percent_genes=10)

ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
solution_str = ''.join(chr(int(gene)) for gene in solution)
print("Passcode cracked:", solution_str)



/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
/usr/local/lib/python3.10/dist-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause t

Passcode cracked: PYTHON


# DEAP

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
import random
from deap import base, creator, tools
import string

target_passcode = "PYTHON345er"
alphabet = string.ascii_letters + string.digits

# Define the fitness function
def fitness(individual):
    return sum(individual[i] == target_passcode[i] for i in range(len(target_passcode))),

# Set up the DEAP framework
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_char", random.choice, alphabet)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_char, n=len(target_passcode))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=5)

def main():
    random.seed(33)
    population = toolbox.population(n=500)

    # Parameters
    prob_crossover = 0.7
    prob_mutation = 0.2
    generations = 500

    for gen in range(generations):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < prob_crossover:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            if random.random() < prob_mutation:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # After crossover and mutation in your genetic algorithm loop
        for ind in offspring:
            if len(ind) != len(target_passcode):
                print("Length mismatch detected after genetic operations.")
                ind[:] = ind[:len(target_passcode)]
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        population[:] = offspring

        # Check for a solution
        fits = [ind.fitness.values[0] for ind in population]

        # if max(fits) == len(target_passcode):
        #     break

    best_ind = tools.selBest(population, 1)[0]
    #print(best_ind)
    #print("Best Individual = ", ''.join(best_ind), ", Fitness = ", best_ind.fitness.values[0])
    print("Best Individual = ", ''.join(str(x) for x in best_ind), ", Fitness = ", best_ind.fitness.values[0])




if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


['P', 'Y', 'T', 'H', 'O', 'N', '3', '4', '5', 'e', 'r']
Best Individual =  PYTHON345er , Fitness =  11.0
